# Generate your requirements.yaml file

This notebook exports the current environment's package requirements to a `requirements.yaml` file. This file is really important as you will need it to upload it together with the notebook into your repository. Please, run the code cell bellow and follow the steps.

> **IMPORANT**: Make sure that you are running this notebook in the same environment you used to run the notebook you are uploading.

In [ ]:
# @markdown Run this cell to choose notebook and extract requirements

# First of all check if ipywidgets is installed, if not through an informative error
try:
    import ipywidgets as widgets
except ImportError:
    raise ImportError("ipywidgets is not installed. Please install it using 'pip install ipywidgets' or 'conda install -c conda-forge ipywidgets'.")

# For the requirements extraction we will need nbformat and pyyaml installed, also check for them
try:
    import nbformat
except ImportError:
    raise ImportError("nbformat is not installed. Please install it using 'pip install nbformat' or 'conda install -c conda-forge nbformat'.")

try:
    import yaml
except ImportError:
    raise ImportError("pyyaml is not installed. Please install it using 'pip install pyyaml' or 'conda install -c conda-forge pyyaml'.")

from IPython.display import display
from pathlib import Path
import importlib.metadata
import subprocess
import contextlib
import importlib
import platform
import nbformat
import yaml
import re
import sys
import os
import io
import html

import_regex = r'^[ \t]*import .*'
from_regex = r'^[ \t]*from .* import .*'

LOCAL_SCAN_EXTENSIONS = {'.py', '.ipynb'}
LOCAL_SCAN_LIMIT = 200  # fail-safe limit to avoid scanning huge trees
LOCAL_SCAN_PREVIEW = 8

def extract_code_import_lines(code):
    list_import_lines = []

    # We are going line by line analyzing them
    lines = code.split('\n')
    for line in lines:
        if re.match(import_regex, line) or re.match(from_regex, line):
            list_import_lines.append(line.strip())
    return list_import_lines

def extract_imported_packages(code):
    list_import_packages = []
    lines = code.split('\n')
    for line in lines:
        line = line.strip()

        # Handle "import X" or "import X, Y" or "import X as Y"
        if re.match(import_regex, line):
            # Extract package name after 'import'
            match = re.search(r'import\s+(.+)', line)
            if match:
                packages = match.group(1).split(',')  # Handle multiple imports
                for pkg in packages:
                    pkg = pkg.split(' as ')[0].strip()  # Remove "as alias"
                    pkg = pkg.split('.')[0]  # Get root package only
                    if pkg:
                        list_import_packages.append(pkg)

        # Handle "from X import Y"
        elif re.match(from_regex, line):
            # Extract module name after 'from'
            match = re.search(r'from\s+([^\s]+)\s+import', line)
            if match:
                module = match.group(1).split('.')[0]  # Get root package
                if module and module != '.':  # Skip relative imports
                    list_import_packages.append(module)

    return list_import_packages

def extract_pip_install(code):
    list_pip_install_lines = []
    lines = code.split('\n')

    for line in lines:
        line = line.strip()

        # Skip empty lines and comments
        if not line or line.startswith('#'):
            continue

        # Check if line starts with ! or %
        if line.startswith('!') or line.startswith('%'):
            # Remove the leading ! or %
            clean_command = line[1:].strip()

            # Verify it contains 'install' keyword
            if 'install' in clean_command:
                # Remove trailing comments if present
                if '#' in clean_command:
                    clean_command = clean_command.split('#')[0].strip()

                list_pip_install_lines.append(clean_command)

    return list_pip_install_lines

def is_builtin_or_stdlib(package_name):
    """Check if a package is a built-in or standard library module."""
    if package_name in sys.builtin_module_names:
        return True

    if hasattr(sys, 'stdlib_module_names') and package_name in sys.stdlib_module_names:
        return True

    # Fallback: common stdlib modules for older Python versions
    stdlib_modules = {
        're', 'sys', 'os', 'platform', 'pathlib', 'json', 'math', 'random',
        'datetime', 'time', 'collections', 'itertools', 'functools', 'operator',
        'pickle', 'csv', 'sqlite3', 'threading', 'subprocess', 'shutil', 'glob',
        'io', 'codecs', 'logging', 'argparse', 'configparser', 'tempfile',
        'urllib', 'http', 'email', 'base64', 'hashlib', 'hmac', 'secrets',
        'struct', 'textwrap', 'string', 'warnings', 'contextlib', 'abc',
        'importlib', 'inspect', 'traceback', 'unittest', 'doctest', 'pdb',
        'profile', 'pstats', 'timeit', 'statistics', 'asyncio', 'concurrent',
        'socket', 'ssl', 'select', 'selectors', 'queue', 'multiprocessing',
        'copy', 'types', 'enum', 'dataclasses', 'typing', 'weakref',
        'array', 'heapq', 'bisect', 'graphlib', 'decimal', 'fractions',
        'numbers', 'cmath', 'zlib', 'gzip', 'bz2', 'lzma', 'tar', 'zipfile',
        'netrc', 'xdrlib', 'plistlib', 'html', 'xml',
        'webbrowser', 'cgi', 'wsgiref', 'uuid', 'socketserver', 'xmlrpc',
    }

    return package_name in stdlib_modules


def is_environment_specific(package_name):
    """Check if a package is environment-specific and shouldn't be in requirements."""
    environment_specific = {
        'google',
        'google.colab',
        'colab',
        'IPython',
        'ipykernel',
        'jupyter',
        'jupyterlab',
    }
    return package_name in environment_specific


def package_exists_on_pypi(package_name):
    """Check if a package exists on PyPI using the PyPI JSON API."""
    try:
        import requests
        response = requests.get(f"https://pypi.org/pypi/{package_name}/json", timeout=2)
        return response.status_code == 200
    except:
        return True


def get_package_version(package_name):
    try:
        module = importlib.import_module(package_name)
        return module.__version__
    except AttributeError:
        try:
            return importlib.metadata.version(package_name)
        except:
            return ""
    except:
        return ""


def get_package_name_for_pip(import_name):
    """
    Convert import name to pip package name.
    E.g., 'docx' -> 'python-docx', 'cv2' -> 'opencv-python'
    """
    known_mappings = {
        'docx': 'python-docx',
        'cv2': 'opencv-python',
        'skimage': 'scikit-image',
        'PIL': 'Pillow',
        'yaml': 'pyyaml',
        'sklearn': 'scikit-learn',
        'bs4': 'beautifulsoup4',
        'dotenv': 'python-dotenv',
        'fpdf': 'fpdf2',
    }

    if import_name in known_mappings:
        return known_mappings[import_name]

    # Try to find via installed distributions
    try:
        for dist in importlib.metadata.distributions():
            try:
                top_level = dist.read_text('top_level.txt')
                if top_level and import_name in top_level.split('\n'):
                    return dist.name
            except:
                pass
    except:
        pass

    # Fallback: assume import name is correct
    return import_name


def load_pip_freeze_requirements():
    mapping = {}
    try:
        result = subprocess.check_output([sys.executable, '-m', 'pip', 'freeze'], text=True)
    except Exception:
        return mapping

    for raw_line in result.splitlines():
        line = raw_line.strip()
        if not line or line.startswith('#'):
            continue

        requirement = line
        key = None

        if line.startswith('-e '):
            entry = line[3:].strip()
            if '#egg=' in entry:
                key = entry.split('#egg=', 1)[1]
            elif ' @ ' in entry:
                key = entry.split(' @ ', 1)[0]
            requirement = entry

        elif ' @ ' in line:
            key = line.split(' @ ', 1)[0]

        elif '==' in line:
            key = line.split('==', 1)[0]

        elif '#egg=' in line:
            key = line.split('#egg=', 1)[1]

        else:
            key = line

        if key:
            mapping[key.strip().lower()] = requirement

    return mapping


def requirement_from_pip_freeze(import_name, pip_freeze_requirements):
    lower_name = import_name.lower()
    if lower_name in pip_freeze_requirements:
        return pip_freeze_requirements[lower_name]

    pip_name = get_package_name_for_pip(import_name)
    if pip_name and pip_name.lower() in pip_freeze_requirements:
        return pip_freeze_requirements[pip_name.lower()]

    return None

def sanitize_path_input(raw):
    """
    Turn a user-pasted path into a safe Path without using `os`.
    - Strips ASCII + smart quotes and surrounding punctuation
    - Expands ~ to the user home directory
    - Replaces common Windows placeholders (%USERPROFILE%, %HOMEPATH%) with the home dir
    - Normalises mixed slashes
    """
    s = str(raw or "").strip()
    if not s:
        return Path("")

    # Remove all kinds of quotes people paste (ASCII and curly/smart quotes)
    s = re.sub(r'[\"\'\u201c\u201d\u2018\u2019]', "", s)

    # Trim accidental leading/trailing punctuation or commas
    s = s.strip(" \t\n\r,;<>")

    # Expand ~ at the start to the user's home dir
    home = Path.home().as_posix()
    if s == "~":
        s = home
    elif s.startswith("~/") or s.startswith("~\\"):
        s = home + s[1:]

    # Common Windows env placeholders -> expand to home (no os.environ access here)
    # This covers the most common pasted forms like %USERPROFILE%\Documents
    s = s.replace("%USERPROFILE%", home)
    s = s.replace("%HOMEPATH%", home)

    # Replace repeated backslash-quote artefacts that sometimes appear from copy/paste
    s = s.replace('\\"', '').replace("\\'", "")

    # Normalise forward/back slashes to the OS-agnostic form; Path() will handle exact OS semantics
    s = s.replace("/", str(Path.home()).startswith("\\") and "\\" or "/") if False else s  # noop kept explicit: Path handles slashes

    # Final strip (in case replacements introduced new edge whitespace/punctuation)
    s = s.strip(" \t\n\r,;<>")

    return Path(s)


def parse_local_path_entries(multiline_text):
    """
    Convert newline-separated user input into Path objects that exist on disk.
    Returns a tuple (paths, errors).
    """
    if not multiline_text:
        return [], []

    parsed_paths = []
    errors = []
    seen = set()

    for raw_line in multiline_text.splitlines():
        trimmed = raw_line.strip()
        if not trimmed:
            continue

        candidate = sanitize_path_input(trimmed).expanduser()
        if not str(candidate):
            continue

        if not candidate.exists():
            errors.append(f"{trimmed} (path not found)")
            continue

        try:
            resolved = candidate.resolve()
        except Exception:
            resolved = candidate

        key = str(resolved)
        if key in seen:
            continue
        seen.add(key)
        parsed_paths.append(resolved)

    return parsed_paths, errors


def expand_local_entries(entries, limit=LOCAL_SCAN_LIMIT):
    """
    Expand a list of file/folder entries into individual files we can scan.
    Returns (files, warnings). Only .py and .ipynb files are considered.
    """
    expanded_files = []
    warnings = []
    seen = set()

    def register_file(file_path):
        try:
            resolved = file_path.resolve()
        except Exception:
            resolved = file_path

        key = str(resolved)
        if key in seen:
            return True

        if limit and len(expanded_files) >= limit:
            return False

        seen.add(key)
        expanded_files.append(resolved)
        return True

    for entry in entries:
        try:
            if entry.is_file():
                if entry.suffix.lower() in LOCAL_SCAN_EXTENSIONS:
                    if not register_file(entry):
                        warnings.append(
                            f"Reached the scan limit ({limit} files). Remaining files were skipped."
                        )
                        break
                else:
                    warnings.append(f"{entry} (unsupported file type)")
            elif entry.is_dir():
                found_any = False
                for suffix in LOCAL_SCAN_EXTENSIONS:
                    for candidate in sorted(entry.rglob(f"*{suffix}")):
                        found_any = True
                        if not register_file(candidate):
                            warnings.append(
                                f"Reached the scan limit ({limit} files). Remaining files were skipped."
                            )
                            return expanded_files, warnings
                if not found_any:
                    warnings.append(f"{entry} (no .py or .ipynb files found)")
            else:
                warnings.append(f"{entry} (not a file or folder)")
        except Exception as exc:
            warnings.append(f"{entry}: {exc}")

    return expanded_files, warnings


def resolve_ipywidgets_version(py_version):
    try:
        py_major, py_minor = map(int, py_version.split(".")[:2])
        
        if py_major >= 3 and py_minor >= 9:
            return "8.1.7"  # Latest stable for modern Python + JupyterLab 4
        elif py_major >= 3 and py_minor >= 7:
            return "8.1.6"  # Good for Python 3.7+
        else:
            return "7.7.2"  # Fallback for older environments
    except Exception:
        return ">=7.0.0"  # Safe fallback

def is_colab():
    return "COLAB_RELEASE_TAG" in os.environ or "COLAB_GPU" in os.environ

def pick_notebook_path_ui_then_run(colab_flag):
    """
    In Colab, ask the user for a notebook path using widgets (recommended: Drive path).
    When a valid path is provided and confirmed, run the extraction.
    """
    mount_title = widgets.HTML(
        "<h2>Step 1: Click the button to mount your Google Drive</h2>"
    )
    mount_btn = widgets.Button(
        description="Mount Google Drive",
        button_style="info",
        layout=widgets.Layout(width="100%")
    )
    mount_status = widgets.HTML("")

    colab_find_notebook = widgets.HTML(
        "<h2>Step 2: Find your notebook</h2>"
        "<ol>"
        "<li>On the left sidebar, click on the folder icon to open the file explorer.</li>"
        "<li>You should see a `drive` folder, otherwise go back to Step 1.</li>"
        "<li>Go to <b>drive</b> > <b>MyDrive</b>. There you will find your Google Drive files.</li>"
        "<li>Locate the notebook you want to extract requirements from. Notebook are stored by default in <b>Colab Notebooks</b> folder.</li>"
        "</ol>"

        "<h4>What if I cannot find my notebook?</h4>"
        "It might happen that your notebook is not stored in Google Drive. <b>Do you have it stored locally on your computer?</b>"
        "<ul>"
        "<li>If <b>yes</b>, drag and drop it in Colab's file explorer for a single use or upload it to Google Drive for persistent storage.</li>"
        "<li>If <b>not</b>, you can download it from wherever you have it stored (e.g., GitHub, email, etc.) and then upload it to Google Drive or directly to Colab.</li>"
        "</ul>"
        "In case the notebook is in another Colab session you can always download it following these steps:"
        "<ol>"
        "<li>Open the Colab session where your notebook is.</li>"
        "<li>On top, go to <b>File</b> > <b>Download</b> > <b>Download .ipynb</b>.</li>"
        "</ol>"
    )
    local_find_notebook = widgets.HTML(
        "<h2>Step 1: Locate your local notebook</h2>"
        "Please make sure you know the full path to your local notebook file (with .ipynb extension)."
    )

    colab_path_title = widgets.HTML(
        "<h2>Step 3: Provide a notebook path</h2>"
        "Right-click on the notebook on Colab's file explorer and select 'Copy path'.<br>"
        "Then paste the full path below and click 'Use this notebook'."
    )
    local_path_title = widgets.HTML(
        "<h2>Step 2: Provide a notebook path</h2>"
        "Please paste the full path to your local notebook below and click 'Use this notebook'."
    )
    path_widget = widgets.Text(
        placeholder="/content/drive/MyDrive/your_notebook.ipynb" if colab_flag else "path/to/your_notebook.ipynb",
        description="Notebook path:",
        layout=widgets.Layout(width="99%"),
        style={'description_width': 'initial'}
    )
    path_status = widgets.HTML("")
    content_container = widgets.VBox(layout=widgets.Layout(width="100%"))

    use_btn = widgets.Button(
        description="Use this notebook",
        button_style="success",
        layout=widgets.Layout(width="99%")
    )

    def on_mount(_):
        try:
            from google.colab import drive
            with contextlib.redirect_stdout(io.StringIO()), contextlib.redirect_stderr(io.StringIO()):
                drive.mount("/content/drive", force_remount=True)
            mount_status.value = "<b style='color:green;'>Drive mounted.</b> Update the path if needed, then click 'Use this notebook'."
        except Exception as e:
            mount_status.value = f"<b style='color:red;'>Drive mount failed:</b> {str(e)}"

    def on_use(_):
        nb_path = sanitize_path_input(path_widget.value).expanduser()

        # Allow folder input: if it's a directory, try to find .ipynb inside it
        if nb_path.exists() and nb_path.is_dir():
            nbs = sorted(nb_path.glob("*.ipynb"))
            if len(nbs) == 1:
                nb_path = nbs[0]
            elif len(nbs) > 1:
                path_status.value = "<b style='color:red;'>Multiple notebooks found in that folder. Please paste the full .ipynb path.</b>"
                content_container.children = ()
                return
            else:
                path_status.value = "<b style='color:red;'>No .ipynb files found in that folder. Please paste a full .ipynb path.</b>"
                content_container.children = ()
                return

        if not nb_path.exists() or nb_path.suffix != ".ipynb":
            if colab_flag:
                path_status.value = (
                    "<b style='color:red;'>Invalid path.</b> Please paste a valid .ipynb path and try again, e.g., "
                    "<code>/content/drive/MyDrive/Colab Notebooks/your_notebook.ipynb</code>"
                )
            else:
                path_status.value = (
                    "<b style='color:red;'>Invalid path.</b> Please paste an absolute path to a valid local .ipynb path and try again. For example: "
                    "<ul>"
                    r"<li><code>C:\Users\YourName\Notebooks\your_notebook.ipynb</code> (Windows)</li>"
                    "<li><code>/home/yourname/notebooks/your_notebook.ipynb</code> (Linux/Mac)</li>"
                    "</ul>"
                )
            content_container.children = ()
            return

        path_status.value = f"<b style='color:green;'>Using:</b> <code>{nb_path}</code>"
        content_container.children = (widgets.HTML("<b>Processing notebook...</b>"),)

        try:
            run_requirements_extraction(nb_path, content_container=content_container)
        except Exception as exc:
            content_container.children = (
                widgets.HTML(
                    "<b style='color:red;'>Failed to extract requirements:</b> "
                    + str(exc)
                ),
            )

    if colab_flag:
        mount_btn.on_click(on_mount)
    use_btn.on_click(on_use)

    if colab_flag:
        layout_children = [
            mount_title,
            mount_btn,
            mount_status,
            colab_find_notebook,
            colab_path_title,
        ]
    else:
        layout_children = [
            local_find_notebook,
            local_path_title,
        ]

    layout_children.extend([path_widget, use_btn, path_status, content_container])
    container = widgets.VBox(layout_children, layout=widgets.Layout(width="100%"))
    display(container)

def run_requirements_extraction(notebook_path: Path, content_container=None):
    def render_section(*objs):
        """Render downstream UI inside a single widget container (or fallback to display)."""
        if content_container is not None:
            content_container.children = tuple(objs)
        else:
            display(*objs)

    # Load pip-freeze mapping (do it here so it reflects the runtime env at click-time)
    pip_freeze_requirements = load_pip_freeze_requirements()

    notebook_name = notebook_path.name

    def process_notebook(additional_entries):
        loading = widgets.HTML("<b>Scanning notebook and selected files...</b>")
        render_section(loading)

        try:
            local_dependency_files, local_scan_warnings = expand_local_entries(
                additional_entries,
                limit=LOCAL_SCAN_LIMIT,
            )
        except Exception as exc:
            render_section(
                widgets.HTML(
                    "<b style='color:red;'>Failed to scan the provided paths:</b> "
                    + html.escape(str(exc))
                )
            )
            return

        try:
            colab_nb = nbformat.read(notebook_path, as_version=4)
        except Exception as exc:
            render_section(
                widgets.HTML(
                    "<b style='color:red;'>Unable to read the selected notebook:</b> "
                    + html.escape(str(exc))
                )
            )
            return

        # Gather imports
        all_imports = []
        pip_install_lines = []

        def collect_from_code(code_blob):
            if not code_blob:
                return
            all_imports.extend(extract_imported_packages(code_blob))
            pip_install_lines.extend(extract_pip_install(code_blob))

        for cell in colab_nb.cells:
            if cell.cell_type == "code":
                collect_from_code(cell.source)

        for local_file in local_dependency_files:
            suffix = local_file.suffix.lower()
            try:
                if suffix == ".ipynb":
                    helper_nb = nbformat.read(local_file, as_version=4)
                    for helper_cell in helper_nb.cells:
                        if helper_cell.cell_type == "code":
                            collect_from_code(helper_cell.source)
                elif suffix == ".py":
                    try:
                        code_blob = local_file.read_text(encoding="utf-8")
                    except UnicodeDecodeError:
                        code_blob = local_file.read_text(encoding="utf-8", errors="ignore")
                    collect_from_code(code_blob)
                else:
                    continue
            except Exception as exc:
                local_scan_warnings.append(f"{local_file}: {exc}")

        # Remove duplicates while preserving order
        all_imports = list(dict.fromkeys(all_imports))
        pip_install_lines = list(dict.fromkeys(pip_install_lines))

        # Process packages to get versions
        versioned_packages = []
        not_found_packages = []
        filtered_packages = []

        for pkg in all_imports:
            normalized_pkg = pkg.strip()

            if is_builtin_or_stdlib(normalized_pkg):
                continue

            if is_environment_specific(normalized_pkg):
                filtered_packages.append(f"{normalized_pkg} (environment-specific)")
                continue

            freeze_requirement = requirement_from_pip_freeze(
                normalized_pkg, pip_freeze_requirements
            )
            pip_name = get_package_name_for_pip(normalized_pkg)

            if freeze_requirement:
                versioned_packages.append(freeze_requirement)
                continue

            try:
                version = get_package_version(normalized_pkg)
            except Exception:
                version = ""

            if not version and pip_name and pip_name != normalized_pkg:
                version = get_package_version(pip_name)

            if pip_name and not package_exists_on_pypi(pip_name):
                filtered_packages.append(f"{pip_name} (not on PyPI)")
                continue

            resolved_name = pip_name or normalized_pkg

            if version:
                versioned_packages.append(f"{resolved_name}=={version}")
            else:
                not_found_packages.append(resolved_name)

        # Get the current Python version
        python_version = platform.python_version()

        def build_local_files_html():
            if not local_dependency_files:
                return ""
            preview_paths = local_dependency_files[:LOCAL_SCAN_PREVIEW]
            value = "<b>Local files scanned:</b><br><ul>"
            for path in preview_paths:
                value += f"<li>{html.escape(str(path))}</li>"
            remaining = len(local_dependency_files) - len(preview_paths)
            if remaining > 0:
                value += f"<li>... and {remaining} more</li>"
            value += "</ul>"
            return value

        def build_local_warnings_html():
            if not local_scan_warnings:
                return ""
            value = "<b>Local scan notes:</b><br><ul>"
            for note in local_scan_warnings:
                value += f"<li>{html.escape(str(note))}</li>"
            value += "</ul>"
            return value

        local_files_html_value = build_local_files_html()
        local_warnings_html_value = build_local_warnings_html()

        def show_requirements_dialog():
            """Display the version info and generate requirements dialog."""
            version_info = widgets.HTML(
                "<h2>Extracted Requirements</h2>"
                "<p>The following packages were detected from the notebook and any optional helper files you provided.</p>"
            )

            if local_files_html_value:
                version_info.value += local_files_html_value

            version_info.value += "<b>Found packages with versions:</b><br>"
            version_info.value += "<ul>"
            for vp in versioned_packages:
                version_info.value += f"<li>{html.escape(vp)}</li>"
            version_info.value += "</ul>"

            if filtered_packages:
                version_info.value += "<b>Filtered packages (excluded from requirements):</b><br>"
                version_info.value += "<ul>"
                for fp in filtered_packages:
                    version_info.value += f"<li>{html.escape(fp)}</li>"
                version_info.value += "</ul>"

            if not_found_packages:
                version_info.value += "<b>Packages not found on your environment or without version info:</b><br>"
                version_info.value += "<ul>"
                for np in not_found_packages:
                    version_info.value += f"<li>{html.escape(np)}</li>"
                version_info.value += "</ul>"

            if local_warnings_html_value:
                version_info.value += local_warnings_html_value

            version_info.value += f"<b>Current Python version:</b> {html.escape(python_version)}<br>"

            # Ask for a notebook description + generate requirements.yaml
            default_notebook_description = f"This notebook '{notebook_name}' is for ..."

            description = widgets.Textarea(
                value=default_notebook_description,
                placeholder='Type something',
                description='Description:',
                disabled=False,
                layout=widgets.Layout(width='99%', height='80px')
            )

            button = widgets.Button(
                description='Generate requirements.yaml',
                disabled=False,
                button_style='success',
                tooltip='Click to generate requirements.yaml file',
                icon='check',
                layout=widgets.Layout(width='99%')
            )
            output = widgets.HTML()

            def on_button_clicked(_):
                output.value = "" # Clear previous output
                
                if not description.value.strip():
                    output.value += "<b style='color:red;'>Please provide a valid description.</b>"
                    return

                # Add ipywidgets with a fixed version if not in the list or replace it if existing
                added_ipywidgets = False
                ipywidget_version = resolve_ipywidgets_version(python_version)
                ipywidget_requirement = f"ipywidgets=={ipywidget_version}"
                for idx, pkg in enumerate(versioned_packages):
                    if pkg.startswith("ipywidgets"):
                        versioned_packages[idx] = ipywidget_requirement
                        added_ipywidgets = True
                        break
                if not added_ipywidgets:
                    versioned_packages.append(ipywidget_requirement)
                else:
                    output.value += f"⚠️ Warning! The ipywidgets version has been overridden to {ipywidget_version}.<br>"

                requirements = {
                    'description': description.value,
                    'python_version': python_version,
                    'dependencies': versioned_packages,
                }

                requirements_path = Path('requirements.yaml')

                with open(requirements_path, 'w') as file:
                    yaml.dump(requirements, file, default_flow_style=False)

                output.value += "<b>Congrats! Your requirements.yaml file has been generated successfully!</b><br>"
                if is_colab():
                    from google.colab import files
                    files.download(requirements_path)
                    output.value += f'Your requirements.yaml file will be automatically downloaded. Additionally, you could find it in <b>{requirements_path.resolve()}</b><br>'
                else:
                    output.value += f'You can find your requirements.yaml file in <a href="{requirements_path.resolve()}" target="_blank" style="color: #0066cc; text-decoration: underline;">{requirements_path.resolve()}</a>.<br>'
                output.value += "Please validate its content, edit if necessary (missed packages or edge cases) and in case you edited validate it using the Requirements Validator notebook."

            button.on_click(on_button_clicked)

            render_section(version_info, description, button, output)

        # Check if there are pip install lines
        if pip_install_lines:
            explanation_html = widgets.HTML(
                "<b>Note:</b> The following pip install commands were detected in the notebook or the scanned helper files.<br>"
                "Please ensure these packages are installed in order to be detected and included in the generated requirements.yaml file.<br>"
                "If you want to install them now, please:"
                "<ol>"
                "<li>Copy and paste the code lines below.</li>"
                "<li>Create a new code cell and paste the code lines there.</li>"
                "<li>Run the cell to install the packages.</li>"
                "<li>Re-run this requirements extraction cell after installation.</li>"
                "</ol>"
                "If you have already installed them, you can ignore this message and click on the 'Continue to generate requirements.yaml' button below."
            )
            pip_installed_textarea = widgets.Textarea(
                value="\n".join([f"!{line}" for line in pip_install_lines]),
                description='Detected pip installs:',
                layout=widgets.Layout(width='99%', height='100px'),
                style={'description_width': 'initial'}
            )
            continue_button = widgets.Button(
                description='Continue to generate requirements.yaml',
                disabled=False,
                button_style='success',
                tooltip='Click to continue',
                icon='check',
                layout=widgets.Layout(width='99%')
            )

            def on_continue_clicked(_):
                show_requirements_dialog()

            continue_button.on_click(on_continue_clicked)

            widgets_to_render = []
            if local_files_html_value:
                widgets_to_render.append(widgets.HTML(local_files_html_value))
            if local_warnings_html_value:
                widgets_to_render.append(widgets.HTML(local_warnings_html_value))
            widgets_to_render.extend([explanation_html, pip_installed_textarea, continue_button])
            render_section(*widgets_to_render)
        else:
            # No pip installs found, show requirements dialog directly
            show_requirements_dialog()

    def show_additional_paths_prompt():
        instructions = widgets.HTML(
            f"<h2>Optional: Include local helper modules</h2>"
            "<p>If your notebook imports functions or classes from local .py files or sibling notebooks, list those paths below (one per line) so we can scan them for dependencies.</p>"
            "<ul>"
            "<li>Paths can be absolute or relative to the current working directory.</li>"
            "<li>Folders are scanned recursively for <code>.py</code> and <code>.ipynb</code> files.</li>"
            f"<li>To keep things fast we scan up to {LOCAL_SCAN_LIMIT} files.</li>"
            "<li>Leave the field empty or click skip if there are no helper modules.</li>"
            "</ul>"
        )
        paths_input = widgets.Textarea(
            placeholder="e.g.\nsrc/helpers.py\ncustom_layers/\n../shared/notebooks/data_loader.ipynb",
            description='Local files/folders:',
            layout=widgets.Layout(width='99%', height='140px'),
            style={'description_width': 'initial'}
        )
        scan_btn = widgets.Button(
            description='Scan paths and continue',
            button_style='info',
            layout=widgets.Layout(width='50%')
        )
        skip_btn = widgets.Button(
            description='Skip (no extra files)',
            layout=widgets.Layout(width='50%')
        )
        status = widgets.HTML()

        def handle_scan(_):
            entries, errors = parse_local_path_entries(paths_input.value)
            if errors:
                message = "<b style='color:red;'>We could not find these paths:</b><ul>"
                for err in errors:
                    message += f"<li>{html.escape(err)}</li>"
                message += "</ul>"
                status.value = message
                return
            status.value = ""
            process_notebook(entries)

        def handle_skip(_):
            status.value = ""
            process_notebook([])

        scan_btn.on_click(handle_scan)
        skip_btn.on_click(handle_skip)

        buttons_box = widgets.HBox(
            [scan_btn, skip_btn],
            layout=widgets.Layout(width='99%', justify_content='space-between')
        )

        render_section(instructions, paths_input, buttons_box, status)

    show_additional_paths_prompt()

# ------------------ ENTRYPOINT ------------------
pick_notebook_path_ui_then_run(is_colab())
